In [1]:
from qdrant_client import QdrantClient
import openai
import os 

In [2]:
openai.__version__

'1.31.0'

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')
client = QdrantClient(url=os.getenv("Qdrant_cluster_url"), 
                      api_key=os.getenv("Qdrant_API_KEY"))
client

In [5]:
## Functions Needed
collection_name = 'Fed_Speeches'

def get_embeddings(text):
    try:
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-3-small"  # Check the latest documentation for available engines
        )
        return response.data[0].embedding  # Extracting the embedding vector
    except Exception as e:
        return str(e)  
    

def search(collection_name, search_vector):
    search_response = client.search(
    collection_name=collection_name,
    query_vector=search_vector,
    limit = 20)
    return search_response
    


In [6]:
search_vector = get_embeddings("The economy is doing bad, there is a lot of inflation!")
search_response = search(collection_name, search_vector)

ids = [hit.id for hit in search_response]
# Assuming texts were stored as payload
contexts = [hit.payload['text'] for hit in search_response]

In [9]:
context = " ".join(contexts)
question = "how is the inflation in the economy?"
prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"

In [10]:
response = openai.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt,
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [11]:
response.choices[0].text

' It is true that inflation is a concern, especially for those in lower income brackets, but the overall economy has been strong and we are currently experiencing lower inflation rates than we have in the past. We are monitoring the situation closely and are continuing to work towards bringing inflation down to a healthier level. While there is still work to be done, the economy itself is not in a bad state.'

In [15]:
response.choices[0].text

' It is true that inflation is a concern, especially for those in lower income brackets, but the overall economy has been strong and we are currently experiencing lower inflation rates than we have in the past. We are monitoring the situation closely and are continuing to work towards bringing inflation down to a healthier level. While there is still work to be done, the economy itself is not in a bad state.'